In [121]:
import torch
torch.cuda.is_available()

True

In [122]:
## get devices 0 and 1
devices = torch.cuda.device_count()
if devices > 1:
    device = '0,1'
else:
    device = '0'
device

'0,1'

In [123]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196  Python-3.8.18 torch-2.1.2+cu118 CUDA:0 (NVIDIA RTX A6000, 49140MiB)
Setup complete  (64 CPUs, 1023.6 GB RAM, 3663.9/16764.0 GB disk)


In [124]:
from ultralytics import YOLO
from IPython.display import display, Image

In [125]:
import os
cwd = os.getcwd()
print(cwd)
if os.path.basename(cwd) == 'datasets':
    cwd = '..'
    os.chdir(cwd)
    cwd = os.getcwd()
elif os.path.basename(cwd) != 'yolov8':
    cwd = '../models/yolov8'
    os.chdir(cwd)
    cwd = os.getcwd()
else:
    pass
print(cwd)

E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8\datasets
E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8


In [126]:
model_path = os.path.join(cwd, 'yolov8x-seg.pt')
model = YOLO(model_path)

In [127]:
from roboflow import Roboflow

In [128]:
datasets = ['sunflowerstarseg', 'starseg']
dataset_code = datasets[0]
dataset_workspace = {
    'sunflowerstarseg': 'sunflowerstarlab',
    'starseg': 'willem-weertman'
}
dataset_workspace = dataset_workspace[dataset_code]
print(f'Dataset code: {dataset_code}, Dataset workspace: {dataset_workspace}')


Dataset code: sunflowerstarseg, Dataset workspace: sunflowerstarlab


In [129]:
version = 4
datasets_root = os.path.join(cwd, 'datasets')
if not os.path.exists(datasets_root):
    os.makedirs(datasets_root)
dataset_location = os.path.join(datasets_root, f'{dataset_code}-{version}')
if not os.path.exists(dataset_location):
    os.makedirs(dataset_location)
print(dataset_location)

E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8\datasets\sunflowerstarseg-4


In [130]:
%cd {datasets_root}

E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8\datasets


In [131]:
# Check if the dataset is already downloaded
if not os.listdir(dataset_location) or len(os.listdir(dataset_location)) == 0:  # If directory is empty
    if dataset_workspace == 'sunflowerstarlab':
        rf = Roboflow(api_key="3DMRGtyRTVJorWA7u61H")
    else:
        rf = Roboflow(api_key="3DMRGtyRTVJorWA7u61H")

    project = rf.workspace(dataset_workspace).project(dataset_code)
    dataset_version = project.version(version)
    dataset = dataset_version.download("yolov8")
else:
    print('Dataset already downloaded')

Dataset already downloaded


In [132]:
## check number of files in dataset_location
len(os.listdir(dataset_location))

6

In [133]:
path_yaml = f'{dataset_location}/data.yaml'
os.path.exists(path_yaml)

True

In [134]:
## open yaml contents and print them
with open(path_yaml, 'r') as file:
    print(file.read())

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Other_species', 'Pycnopodia_helianthoides']

roboflow:
  workspace: sunflowerstarlab
  project: sunflowerstarseg
  version: 4
  license: CC BY 4.0
  url: https://universe.roboflow.com/sunflowerstarlab/sunflowerstarseg/dataset/4


In [135]:
## default dataset paths are..
## test: ../test/images
## train: StarSeg-24/train/images
## val: StarSeg-24/valid/images
## this is a bug, we need to change train and val to have a similar format as test
## they need to be overwritten like..
## train: ../train/images
## val: ../valid/images

## overwrite yaml
with open(path_yaml, 'r') as file:
    filedata = file.read()
    filedata = filedata.replace('test: ../test/images', 'test: ../test/images')
    filedata = filedata.replace(f'train: StarSeg-{version}/train/images', 'train: ../train/images')
    filedata = filedata.replace(f'val: StarSeg-{version}/valid/images', 'val: ../valid/images')

with open(path_yaml, 'w') as file:
    file.write(filedata)

In [136]:
## open yaml contents and print them
with open(path_yaml, 'r') as file:
    print(file.read())

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Other_species', 'Pycnopodia_helianthoides']

roboflow:
  workspace: sunflowerstarlab
  project: sunflowerstarseg
  version: 4
  license: CC BY 4.0
  url: https://universe.roboflow.com/sunflowerstarlab/sunflowerstarseg/dataset/4


In [137]:
if os.getcwd() != os.path.basename(model_path):
    os.chdir(os.path.dirname(model_path))
    print(os.getcwd())

E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8


In [138]:
import datetime
project_date = datetime.datetime.now().strftime('%Y%m%d')
project_name = f'{dataset_code}-{version}-{project_date}'
print(project_name)

sunflowerstarseg-4-20240402


In [139]:
batch_size = 64
imgsz = 640
epochs = 300
workers = 32

data_yaml_location = os.path.join(dataset_location, 'data.yaml')
model.train(model=model_path, data=data_yaml_location, task='segment',
            project=project_name,
            epochs=epochs, batch=batch_size, imgsz=imgsz,
            device=device, workers=workers,
            verbose=True)

New https://pypi.org/project/ultralytics/8.1.42 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.8.18 torch-2.1.2+cu118 CUDA:0 (NVIDIA RTX A6000, 49140MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 49140MiB)
engine\trainer: task=segment, mode=train, model=E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8\yolov8x-seg.pt, data=E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8\datasets\sunflowerstarseg-4\data.yaml, epochs=300, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0,1, workers=32, project=sunflowerstarseg-4-20240402, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hyb

In [140]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.0.196  Python-3.8.18 torch-2.1.2+cu118 CUDA:0 (NVIDIA RTX A6000, 49140MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 49140MiB)
YOLOv8x-seg summary (fused): 295 layers, 71722582 parameters, 0 gradients, 343.7 GFLOPs
val: Scanning E:\SunflowerStarArchive\sunflower-star-archive-tools\models\yolov8\datasets\SunflowerStarSeg-4\valid\labels.cache... 947 images, 0 backgrounds, 0 corrupt: 100%|██████████| 947/947 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:22<00:00,  1.53s/it]
                   all        947       1657      0.961      0.899      0.949      0.857      0.962      0.884      0.939      0.766
         Other_species        947        974      0.949      0.855      0.925      0.851      0.959      0.849      0.926      0.791
Pycnopodia_helianthoides        947        683      0.973      